# Goal of notebook : 

1. Load Lyrics with genre
2. Clean text in parenthesis from lyrics
3. Create Fasttext vectors and read them into dataframe
4. Create column with indicator of last word of line
5. Create column with type of word (noum, verb, etc)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## 1. Load Lyrics with genre

In [33]:
filepath = "../augment db/full_lyrics.csv"
lyrics_df = pd.read_csv(filepath)
lyrics_df.head()

/Users/emma/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,artist,genre,lang,lyrics,title
0,beyonce-knowles,pop,NaN,"Oh baby, how you doing?\r\r\nYou know I'm gonn...",ego-remix
1,beyonce-knowles,pop,NaN,"playin' everything so easy,\r\r\nit's like you...",then-tell-me
2,beyonce-knowles,pop,NaN,If you search\r\r\nFor tenderness\r\r\nIt isn'...,honesty
3,beyonce-knowles,pop,NaN,"Oh oh oh I, oh oh oh I\r\r\n[Verse 1:]\r\r\nIf...",you-are-my-rock
4,beyonce-knowles,pop,NaN,"Party the people, the people the party it's po...",black-culture


In [34]:
lyrics_df = lyrics_df.drop(['lang'], axis = 1)

In [35]:
lyrics_df.head()

,artist,genre,lyrics,title
0,beyonce-knowles,pop,"Oh baby, how you doing?\r\r\nYou know I'm gonn...",ego-remix
1,beyonce-knowles,pop,"playin' everything so easy,\r\r\nit's like you...",then-tell-me
2,beyonce-knowles,pop,If you search\r\r\nFor tenderness\r\r\nIt isn'...,honesty
3,beyonce-knowles,pop,"Oh oh oh I, oh oh oh I\r\r\n[Verse 1:]\r\r\nIf...",you-are-my-rock
4,beyonce-knowles,pop,"Party the people, the people the party it's po...",black-culture


## 2. Clean text in parenthesis from lyrics

In [36]:
sample_df = lyrics_df.sample(n=20)
for i, s in sample_df.iterrows():
    print(s['lyrics'])




































Let us all go stare at boobs
























































Stop why you doing that?




















She's my Filipino baby...
My plain Jane never wear no lipstick or paint
You know she'll never try to be what she ain't
Come early in the morning you can hear the fella's call (woo)

Plain Jane won't you come out tonight so we can have a ball
Plain Jane won't you come out tonight, come out tonight, come out tonight
Plain Jane won't you come out tonight, spend some time with me

Her mouth's a little too wide and her nose a wee bit long
And when you take a good look at her her shape seems all wrong
But she got this certain something that's so hard to define
And I wish that certain something was mine, all mine, all mine

And old Plain Jane won't you come out tonight, come out tonight, come out tonight
Plain Jane won't you come out tonight, spend some time with me

I used to go with glamour girls to worship day and night
But Pla

### Remove [<30 chars] and (<30 chars) and x(digit)

This information gives structure for the singer but not usefull for lyrics. This is [Chorus], choir in parentheses, x2 etc. <br>
To test regex expressions : https://regex101.com

In [37]:
import string
import re

In [38]:
regex_bracket = re.compile("\[(.*?)\]")
regex_parentheses = re.compile("\((.*?)\)")
regex_curly_bracket = re.compile("\{(.*?)\}")
regex_timesx = re.compile("x[0-9]")

def clean_structure_words(l):
    res= regex_bracket.sub('', l)
    res = regex_parentheses.sub('', res)
    res = regex_curly_bracket.sub('', res)
    res = regex_timesx.sub('', res)
    return res

In [8]:
clean_structure_words("Hello, my. name is +emma* (nice)[to]meet -x3--")

'Hello, my. name is +emma* meet ---'

In [39]:
lyrics_df['lyrics'] = lyrics_df['lyrics'].apply(lambda x : clean_structure_words(x))

In [40]:
lyrics_df.head()

,artist,genre,lyrics,title
0,beyonce-knowles,pop,"Oh baby, how you doing?\r\r\nYou know I'm gonn...",ego-remix
1,beyonce-knowles,pop,"playin' everything so easy,\r\r\nit's like you...",then-tell-me
2,beyonce-knowles,pop,If you search\r\r\nFor tenderness\r\r\nIt isn'...,honesty
3,beyonce-knowles,pop,"Oh oh oh I, oh oh oh I\r\r\n\r\r\nIf I wrote a...",you-are-my-rock
4,beyonce-knowles,pop,"Party the people, the people the party it's po...",black-culture


### Remove . , ! ? ... ---
- We want to keep the * as this represents some swear words in english
- We want to keep the ' as this is part of many words in english, especially when words are cropped, ie 'Cause 
- Otherwise punctuation is not meaningful

In [11]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [12]:
string.printable

'0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ \t\n\r\x0b\x0c'

In [41]:
#punctuation_to_remove = '!#$%\+,-./:;<=>?@\\^_`|~()[]{}'
#regex_punct = re.compile('[%s]' % re.escape(punctuation_to_remove))
chars_to_keep = '0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ\'* \n'

def clean_punctuation(l):
    #res = regex_punct.sub('', l)
    res = re.sub(' +', ' ', l)#res)
    res = ''.join(char for char in res if char in chars_to_keep)
    return res

In [15]:
repr(clean_punctuation("Hello,     my.   name I'm (coucou) is +emma* meet ---"))

'"Hello my name I\'m coucou is emma* meet "'

In [42]:
lyrics_df['lyrics'] = lyrics_df['lyrics'].apply(lambda x : clean_punctuation(x))

In [43]:
lyrics_df.head()

,artist,genre,lyrics,title
0,beyonce-knowles,pop,Oh baby how you doing\nYou know I'm gonna cut ...,ego-remix
1,beyonce-knowles,pop,playin' everything so easy\nit's like you seem...,then-tell-me
2,beyonce-knowles,pop,If you search\nFor tenderness\nIt isn't hard t...,honesty
3,beyonce-knowles,pop,Oh oh oh I oh oh oh I\n\nIf I wrote a book abo...,you-are-my-rock
4,beyonce-knowles,pop,Party the people the people the party it's pop...,black-culture


## 3. Create Fasttext vectors and read them into dataframe

In [44]:
# First create a txt file with the lyrics
f = open("lyrics.txt", "w+")
for i, r in lyrics_df.iterrows():
    f.write(r['lyrics'])
    
f.close()

https://fasttext.cc/docs/en/unsupervised-tutorial.html

In [45]:
# Generate embeddings
#! ./fasttext skipgram -input lyrics.txt -output model_lyrics

Read 82M words
Number of words:  125703
Number of labels: 0


Progress: 13.5%  words/sec/thread: 133902  lr: 0.043244  loss: 1.823352  eta: 0h3m 

Progress: 18.1%  words/sec/thread: 133978  lr: 0.040972  loss: 1.818897  eta: 0h3m   words/sec/thread: 133917  lr: 0.043223  loss: 1.823420  eta: 0h3m   lr: 0.043215  loss: 1.823393  eta: 0h3m   words/sec/thread: 133930  lr: 0.043212  loss: 1.823408  eta: 0h3m   words/sec/thread: 133912  lr: 0.043201  loss: 1.823351  eta: 0h3m 1.823177  eta: 0h3m gress: 13.6%  words/sec/thread: 133902  lr: 0.043181  loss: 1.823286  eta: 0h3m   lr: 0.043145  loss: 1.822866  eta: 0h3m 3m   words/sec/thread: 133948  lr: 0.043113  loss: 1.822445  eta: 0h3m m 1.821728  eta: 0h3m   words/sec/thread: 133906  lr: 0.043059  loss: 1.821549  eta: 0h3m 13.9%  words/sec/thread: 133923  lr: 0.043037  loss: 1.821278  eta: 0h3m   words/sec/thread: 133913  lr: 0.043009  loss: 1.820722  eta: 0h3m 0.042962  loss: 1.820333  eta: 0h3m   words/sec/thread: 133859  lr: 0.042951  loss: 1.820246  eta: 0h3m 1.819971  eta: 0h3m   words/sec/thread: 133916  lr: 0.042920  loss: 1.819542  eta: 0h3m   eta: 0h3m   words/sec/thread: 133

Progress: 22.4%  words/sec/thread: 133513  lr: 0.038801  loss: 1.826023  eta: 0h3m   words/sec/thread: 133980  lr: 0.040965  loss: 1.818916  eta: 0h3m   words/sec/thread: 133973  lr: 0.040954  loss: 1.818792  eta: 0h3m   words/sec/thread: 133960  lr: 0.040930  loss: 1.818576  eta: 0h3m   words/sec/thread: 133961  lr: 0.040920  loss: 1.818602  eta: 0h3m   words/sec/thread: 133962  lr: 0.040914  loss: 1.818567  eta: 0h3m   words/sec/thread: 133945  lr: 0.040902  loss: 1.818581  eta: 0h3m   words/sec/thread: 133902  lr: 0.040874  loss: 1.818421  eta: 0h3m   words/sec/thread: 133898  lr: 0.040870  loss: 1.818272  eta: 0h3m   words/sec/thread: 133875  lr: 0.040845  loss: 1.818129  eta: 0h3m   words/sec/thread: 133849  lr: 0.040828  loss: 1.818113  eta: 0h3m 0.040820  loss: 1.818117  eta: 0h3m gress: 18.4%  words/sec/thread: 133843  lr: 0.040799  loss: 1.817418  eta: 0h3m 3m   words/sec/thread: 133821  lr: 0.040783  loss: 1.817361  eta: 0h3m   words/sec/thread: 133815  lr: 0.040768  loss: 1.

Progress: 25.6%  words/sec/thread: 133341  lr: 0.037184  loss: 1.821525  eta: 0h3m   words/sec/thread: 133507  lr: 0.038794  loss: 1.825965  eta: 0h3m   words/sec/thread: 133500  lr: 0.038784  loss: 1.825757  eta: 0h3m   words/sec/thread: 133502  lr: 0.038773  loss: 1.825733  eta: 0h3m 0.038756  loss: 1.825500  eta: 0h3m   words/sec/thread: 133487  lr: 0.038746  loss: 1.825500  eta: 0h3m   words/sec/thread: 133480  lr: 0.038733  loss: 1.825340  eta: 0h3m   words/sec/thread: 133477  lr: 0.038729  loss: 1.825243  eta: 0h3m   loss: 1.825160  eta: 0h3m .6%  words/sec/thread: 133472  lr: 0.038709  loss: 1.824752  eta: 0h3m   words/sec/thread: 133471  lr: 0.038700  loss: 1.824534  eta: 0h3m   words/sec/thread: 133474  lr: 0.038693  loss: 1.824320  eta: 0h3m   words/sec/thread: 133481  lr: 0.038688  loss: 1.824253  eta: 0h3m 0.038663  loss: 1.823908  eta: 0h3m   words/sec/thread: 133488  lr: 0.038658  loss: 1.823909  eta: 0h3m   words/sec/thread: 133505  lr: 0.038629  loss: 1.823932  eta: 0h3

...

Progress: 85.6%  words/sec/thread: 130647  lr: 0.007176  loss: 1.663486  eta: 0h0m ss: 1.671483  eta: 0h0m   words/sec/thread: 130459  lr: 0.008908  loss: 1.671488  eta: 0h0m 1.671470  eta: 0h0m 2.2%  words/sec/thread: 130463  lr: 0.008888  loss: 1.671517  eta: 0h0m   words/sec/thread: 130464  lr: 0.008880  loss: 1.671484  eta: 0h0m 0.008869  loss: 1.671481  eta: 0h0m 2.3%  words/sec/thread: 130463  lr: 0.008859  loss: 1.671449  eta: 0h0m ss: 1.671368  eta: 0h0m sec/thread: 130465  lr: 0.008837  loss: 1.671331  eta: 0h0m   loss: 1.671299  eta: 0h0m   words/sec/thread: 130466  lr: 0.008823  loss: 1.671246  eta: 0h0m 0.008815  loss: 1.671218  eta: 0h0m   words/sec/thread: 130465  lr: 0.008807  loss: 1.671209  eta: 0h0m 0.008798  loss: 1.671168  eta: 0h0m   words/sec/thread: 130468  lr: 0.008792  loss: 1.671126  eta: 0h0m 1.671072  eta: 0h0m thread: 130473  lr: 0.008770  loss: 1.671037  eta: 0h0m ss: 1.671021  eta: 0h0m   words/sec/thread: 130478  lr: 0.008752  loss: 1.670992  eta: 0h0m 0

Progress: 89.4%  words/sec/thread: 130871  lr: 0.005288  loss: 1.655255  eta: 0h0m h0m lr: 0.007145  loss: 1.663205  eta: 0h0m a: 0h0m 48  lr: 0.007123  loss: 1.663049  eta: 0h0m rds/sec/thread: 130649  lr: 0.007114  loss: 1.662932  eta: 0h0m   words/sec/thread: 130650  lr: 0.007101  loss: 1.662858  eta: 0h0m 0.007090  loss: 1.662802  eta: 0h0m 5.8%  words/sec/thread: 130652  lr: 0.007083  loss: 1.662733  eta: 0h0m ss: 1.662653  eta: 0h0m sec/thread: 130657  lr: 0.007059  loss: 1.662594  eta: 0h0m   loss: 1.662584  eta: 0h0m ad: 130659  lr: 0.007038  loss: 1.662544  eta: 0h0m 6.0%  words/sec/thread: 130657  lr: 0.007025  loss: 1.662507  eta: 0h0m   words/sec/thread: 130656  lr: 0.007011  loss: 1.662482  eta: 0h0m 1.662437  eta: 0h0m ad: 130656  lr: 0.006990  loss: 1.662389  eta: 0h0m  lr: 0.006980  loss: 1.662297  eta: 0h0m ss: 86.1%  words/sec/thread: 130657  lr: 0.006970  loss: 1.662275  eta: 0h0m 4  eta: 0h0m  130660  lr: 0.006945  loss: 1.662124  eta: 0h0m 62085  eta: 0h0m ead: 130

Progress: 93.2%  words/sec/thread: 131126  lr: 0.003423  loss: 1.645956  eta: 0h0m 0.005262  loss: 1.655158  eta: 0h0m 9.5%  words/sec/thread: 130883  lr: 0.005248  loss: 1.655103  eta: 0h0m 1.655031  eta: 0h0m 9.6%  words/sec/thread: 130892  lr: 0.005225  loss: 1.654990  eta: 0h0m   eta: 0h0m   words/sec/thread: 130897  lr: 0.005203  loss: 1.654797  eta: 0h0m 1.654652  eta: 0h0m   words/sec/thread: 130901  lr: 0.005185  loss: 1.654546  eta: 0h0m h0m ad: 130911  lr: 0.005156  loss: 1.654155  eta: 0h0m   words/sec/thread: 130917  lr: 0.005142  loss: 1.654048  eta: 0h0m 0.005132  loss: 1.654011  eta: 0h0m 9.8%  words/sec/thread: 130920  lr: 0.005121  loss: 1.653946  eta: 0h0m ss: 1.653893  eta: 0h0m s: 89.8%  words/sec/thread: 130920  lr: 0.005102  loss: 1.653849  eta: 0h0m ad: 130926  lr: 0.005088  loss: 1.653723  eta: 0h0m 9.8%  words/sec/thread: 130926  lr: 0.005076  loss: 1.653661  eta: 0h0m   words/sec/thread: 130929  lr: 0.005066  loss: 1.653656  eta: 0h0m   words/sec/thread: 13093

Progress: 96.9%  words/sec/thread: 131461  lr: 0.001574  loss: 1.638938  eta: 0h0m 3.2%  words/sec/thread: 131125  lr: 0.003404  loss: 1.645869  eta: 0h0m   words/sec/thread: 131124  lr: 0.003391  loss: 1.645854  eta: 0h0m 0.003386  loss: 1.645833  eta: 0h0m   words/sec/thread: 131123  lr: 0.003377  loss: 1.645764  eta: 0h0m 0.003368  loss: 1.645720  eta: 0h0m thread: 131121  lr: 0.003362  loss: 1.645656  eta: 0h0m a: 0h0m   loss: 1.645555  eta: 0h0m 131122  lr: 0.003322  loss: 1.645495  eta: 0h0m ad: 131121  lr: 0.003298  loss: 1.645404  eta: 0h0m h0m lr: 0.003277  loss: 1.645289  eta: 0h0m a: 0h0m   words/sec/thread: 131131  lr: 0.003257  loss: 1.645191  eta: 0h0m 1.645143  eta: 0h0m 3.5%  words/sec/thread: 131134  lr: 0.003234  loss: 1.645117  eta: 0h0m   words/sec/thread: 131138  lr: 0.003225  loss: 1.645088  eta: 0h0m ad: 131138  lr: 0.003214  loss: 1.645047  eta: 0h0m 3.6%  words/sec/thread: 131140  lr: 0.003200  loss: 1.644938  eta: 0h0m a: 0h0m 41  lr: 0.003174  loss: 1.644776 

Progress: 100.0%  words/sec/thread: 131665  lr: 0.000000  loss: 1.633493  eta: 0h0m .638976  eta: 0h0m thread: 131462  lr: 0.001549  loss: 1.638937  eta: 0h0m a: 0h0m sec/thread: 131470  lr: 0.001528  loss: 1.638835  eta: 0h0m ad: 131470  lr: 0.001519  loss: 1.638762  eta: 0h0m ad: 131472  lr: 0.001509  loss: 1.638710  eta: 0h0m h0m   words/sec/thread: 131474  lr: 0.001490  loss: 1.638647  eta: 0h0m 1.638613  eta: 0h0m ad: 131471  lr: 0.001472  loss: 1.638583  eta: 0h0m   words/sec/thread: 131469  lr: 0.001464  loss: 1.638581  eta: 0h0m   words/sec/thread: 131468  lr: 0.001457  loss: 1.638557  eta: 0h0m 1.638534  eta: 0h0m ad: 131464  lr: 0.001442  loss: 1.638512  eta: 0h0m   words/sec/thread: 131467  lr: 0.001430  loss: 1.638499  eta: 0h0m 0.001421  loss: 1.638488  eta: 0h0m h0m thread: 131468  lr: 0.001408  loss: 1.638486  eta: 0h0m /thread: 131468  lr: 0.001399  loss: 1.638446  eta: 0h0m   words/sec/thread: 131470  lr: 0.001390  loss: 1.638386  eta: 0h0m 1.638329  eta: 0h0m 7.3%  wo

In [46]:
# To check first words : The first line is a header containing the number of words and the dimensionality of the vectors. 
# The subsequent lines are the word vectors for all words in the vocabulary, sorted by decreasing frequency.
#! head -n 4 model_lyrics.vec

125703 100
</s> -0.012681 0.039548 0.12973 -0.13181 0.2192 0.006487 -0.069694 0.11984 0.021715 0.1954 -0.070876 0.050559 0.040476 -0.14598 0.21885 -0.20444 -0.14007 0.056377 -0.21201 -0.14093 -0.19119 -0.20454 0.21292 -0.22055 0.22079 0.010239 -0.024036 -0.13012 -0.19501 0.039349 -0.088207 -0.12589 0.029011 0.13177 -0.04941 -0.16296 -0.0151 -0.23948 -0.12409 -0.11343 0.010718 0.083172 -0.16852 0.28312 0.075389 -0.31338 -0.20002 -0.1016 -0.18516 0.01608 -0.20757 0.24317 0.066537 -0.050611 -0.15119 0.054165 -0.13907 0.28233 0.082253 0.067789 0.24579 0.022096 -0.038027 -0.064011 -0.13251 0.043089 -0.22678 0.05545 0.10473 -0.18219 -0.080301 0.14894 0.06075 -0.02897 -0.016911 -0.29241 0.28462 0.096767 -0.086651 0.19748 -0.049296 -0.019989 -0.0040943 -0.21574 0.31605 -0.12374 0.27784 0.20987 -0.077757 0.26502 -0.12618 -0.091735 0.14919 -0.23601 -0.37268 -0.34082 0.094895 0.15425 -0.17881 0.010367 
the -0.041153 -0.048301 0.33608 -0.21116 0.065999 0.25024 0.010657 0.19707 0.057401 0.11175 -0.

In [47]:
import sys
import codecs

In [48]:
# Load embeddings
def load_embeddings(file_name):
    with codecs.open(file_name, 'r', 'utf-8') as f_in:
        lines = f_in.readlines()
        lines = lines[1:]
        vocabulary, wv = zip(*[line.strip().split(' ', 1) for line in lines])
    wv = np.loadtxt(wv)
    return wv, vocabulary

In [49]:
word_embeddings, vocabulary = load_embeddings('model_lyrics.vec')

In [21]:
vocabulary = list(vocabulary)

In [53]:
"eversince" in vocabulary

False

In [51]:
len(vocabulary)

125703

In [52]:
word_embeddings.shape

(125703, 100)

In [106]:
def get_embedding_for_missing_word(w):
    data = ! echo $w | ./fasttext print-word-vectors model_lyrics.bin
    emb = np.zeros(100)
    for i, x in enumerate(data[0].split( )[1:]):
        try:
            emb[i] = float(x)
        except : 
            continue
            #print(data[0])
    return emb

In [92]:
get_embedding_for_missing_word("eversince")

array([ 0.15354  , -0.082218 , -0.2494   , -0.39903  , -0.053004 ,
       -0.20358  ,  0.46507  , -0.48215  ,  0.031456 ,  0.29864  ,
       -0.45612  , -0.52815  ,  0.42328  ,  0.48587  , -0.13842  ,
       -0.027078 , -0.29471  ,  0.31589  , -0.052668 , -0.17239  ,
       -0.062075 , -0.1166   ,  0.062651 , -0.36563  ,  0.59316  ,
       -0.11488  ,  0.31329  ,  0.0054363, -0.22374  ,  0.19368  ,
        0.46279  , -0.13704  , -0.48086  , -0.21136  , -0.41806  ,
        0.096683 ,  0.17842  , -0.12313  , -0.47596  , -0.11186  ,
       -0.093677 ,  0.078458 , -0.11241  , -0.20531  ,  0.28876  ,
        0.25704  , -0.049918 ,  0.54401  ,  0.10132  ,  0.062308 ,
        0.31098  , -0.068861 ,  0.24074  , -0.29187  , -0.2174   ,
       -0.22046  ,  0.075688 ,  0.088568 , -0.25472  ,  0.24982  ,
        0.02957  , -0.011789 , -0.081646 ,  0.031771 , -0.34703  ,
       -0.34078  , -0.24647  , -0.078897 , -0.26882  ,  0.25468  ,
       -0.049018 ,  0.31585  , -0.075016 , -0.0050666, -0.1406

## 4. Create column with indicator of last word of line

In [27]:
test_l = lyrics_df.loc[190]['lyrics']
test_l

"You a bad girl and your friends bad too\nOh you got the swag sauce you dripping swagu\nYou a bad girl and your friends bad too\nOh you got the swag sauce you dripping swagu\n\nI may be young but I'm ready\nTo give you all my love\nI told my girls you can get it\nDon't slow it down just let it go\nSo in love I'll give it all away\nJust don't tell nobody tomorrow\nSo tonight I'll do it every way\nSpeakers knocking til the morning lights\nChorus\nWe like to party hey hey hey hey hey hey\n\nWe like to party hey hey hey hey hey hey\n\nYour touch is driving me crazy\nI can't explain the way I feel\nTop down with the radio on\nAnd the night belongs to us\nJust hold me close don't let me go\nSo in love I don't care what they say\nI don't care if they're talking tomorrow\nCause tonight is the night oh oh oh\nThat I give you everything\nMusic knocking til the morning light\nCause we like to party\nSet the scene 3000 degrees\nAin't worried 'bout them fuck niggas over there\nBut they worried 'bou

In [55]:
def generate_embedding_for_lyrics(l):
    #print(l)
    sentences = l.split('\n')
    embedding = []
    #missing_words = []
    for s in sentences:
        words = s.split(' ')
        #print(repr(words))
        for i, w in enumerate(words):
            if len(w)>0:
                try : 
                    idx = vocabulary.index(w)
                    vector = word_embeddings[idx]
                except ValueError : 
                    #print("Word not found :", repr(w))
                    #missing_words.append(w)
                    vector = get_embedding_for_missing_word(w)
                if i == (len(words)-1) :
                    vector = np.append(vector, 1)
                else :
                    vector = np.append(vector, 0)
                vector = np.append(w, vector)
                embedding.append(vector)
                
    return embedding

In [ ]:
generate_embedding_for_lyrics("Oh baby how you doing\nYou know I'm gonna cut right to the chase")

In [ ]:
lyrics_df['embedding'] = lyrics_df['lyrics'][:30].apply(lambda x : generate_embedding_for_lyrics(x))

echo "eversince" | ./fasttext print-word-vectors model_lyrics.bin
eversince 0.26688 -0.1294 0.21473 -0.0022411 -0.21671 0.032355 -0.18573 0.14012 0.26681 0.37774 -0.33736 0.26381 -0.053749 0.37619 -0.21128 -0.056934 0.20461 0.33779 -0.10611 0.021266 -0.17509 0.27 0.41086 0.13709 0.16705 -0.60041 0.38402 0.087207 -0.33848 0.19228 -0.27138 -0.30764 -0.2702 -0.30729 -0.72478 0.16182 0.11382 0.19189 -0.27027 0.27689 0.039753 -0.21678 -0.4436 0.24418 0.12741 0.21971 0.014112 0.11523 0.00034974 0.24912 -0.62545 -0.04865 -0.32367 -0.38435 -0.058886 -0.13295 0.48399 0.49655 0.34541 0.65489 -0.18568 -0.076875 0.087327 -0.0079827 -0.073017 -0.046165 0.009111 -0.024724 -0.44245 0.092923 0.33201 -0.54304 -0.22664 0.41005 0.14919 -0.30418 -0.13195 -0.10742 -0.18871 0.14914 0.15383 -0.30302 0.362 0.26775 0.11757 0.057333 0.38091 0.095017 -0.10443 0.13355 0.64025 0.070291 0.045029 -0.053138 0.19487 -0.062763 0.19003 0.32966 0.049617 0.207

In [97]:
lyrics_df.head()

,song,year,artist,genre,lyrics,language,embedding
0,ego-remix,2009,beyonce-knowles,pop,Oh baby how you doing\nYou know Im gonna cut r...,en,"[[Oh, 0.11909, 0.08231, -0.28486, 0.23137, 0.1..."
1,then-tell-me,2009,beyonce-knowles,pop,playin everything so easy\nits like you seem s...,en,"[[playin, 0.5371, -0.097827, 0.68525, -0.10739..."
2,honesty,2009,beyonce-knowles,pop,If you search\nFor tenderness\nIt isnt hard to...,en,"[[If, -0.3726, 0.33803, -0.36677, -0.29783, 0...."
3,you-are-my-rock,2009,beyonce-knowles,pop,Oh oh oh I oh oh oh I\n\nIf I wrote a book abo...,en,"[[Oh, 0.11909, 0.08231, -0.28486, 0.23137, 0.1..."
4,black-culture,2009,beyonce-knowles,pop,Party the people the people the party its popp...,en,"[[Party, 0.28332, 0.68937, -0.4371, -0.084831,..."


## 5. Create column with type of word (noum, verb, etc)

In [57]:
import spacy

In [58]:
en_nlp = spacy.load('en')

ADJ: adjective ADP: adposition ADV: adverb AUX: auxiliary verb CONJ: coordinating conjunction DET: determiner INTJ: interjection NOUN: noun NUM: numeral PART: particle PRON: pronoun PROPN: proper noun PUNCT: punctuation SCONJ: subordinating conjunction SYM: symbol VERB: verb X: other

In [71]:
def generate_pos_for_lyrics(l):
    doc = en_nlp(l)
    embedding = []
    for i in range(len(doc)):
        if doc[i].pos_ != 'SPACE':
            print(doc[i], doc[i].pos_)
            embedding.append(doc[i].pos_)
    return embedding

In [72]:
generate_pos_for_lyrics("Oh baby how you doing\nYou know I'm gonna cut right to the chase")

Oh INTJ
baby NOUN
how ADV
you PRON
doing VERB
You PRON
know VERB
I PRON
'm VERB
gon VERB
na PART
cut VERB
right ADV
to ADP
the DET
chase NOUN


['INTJ',
 'NOUN',
 'ADV',
 'PRON',
 'VERB',
 'PRON',
 'VERB',
 'PRON',
 'VERB',
 'VERB',
 'PART',
 'VERB',
 'ADV',
 'ADP',
 'DET',
 'NOUN']

In [90]:
def generate_pos_for_word(w):
    doc = en_nlp(w)
    #print(doc[0], doc[0].pos_)
    return doc[0].pos_

In [91]:
generate_pos_for_word('Oh')

'INTJ'

In [94]:
pos_to_idx = {'ADJ': 0, 'ADP': 1, 'ADV':2,'AUX': 3,'CONJ': 4,'CCONJ': 5,'DET': 6, 'INTJ': 7, 'NOUN': 8,
              'NUM': 9, 'PART': 10,'PRON':11, 'PROPN': 12, 'PUNCT': 13, 'SCONJ': 14, 'SYM': 15, 
              'VERB': 16, 'X': 17}

In [67]:
"""def merge_embedding_pos(l):
    embedding = generate_embedding_for_lyrics(l)
    pos = generate_pos_for_lyrics(l)
    #print(len(embedding))
    for i in range(len(embedding)):
        bin_vector = np.zeros(18)
        bin_vector[pos_to_idx[pos[i]]] = 1
        embedding[i] = np.append(embedding[i], bin_vector)
    return embedding"""

In [108]:
def merge_embedding_pos(l):
    embedding = generate_embedding_for_lyrics(l)
    for i in range(len(embedding)):
        w = embedding[i][0]
        #print(w)
        if w != '\n':
            pos = generate_pos_for_word(str(w))
            bin_vector = np.zeros(18)
            bin_vector[pos_to_idx[pos]] = 1
            embedding[i] = np.append(embedding[i], bin_vector)
    return embedding

In [109]:
merge_embedding_pos("Oh baby how you doing\nYou know I'm gonna cut right to the chase")

[array(['Oh', '-0.26756', '0.16066', '-0.026644', '-0.061647', '0.12856',
        '-0.16667', '-0.31698', '0.21519', '0.087186', '0.70611',
        '0.097478', '-0.017475', '0.13562', '-0.34733', '0.12901',
        '0.060213', '0.0038623', '-0.003505', '-0.309', '-0.1214',
        '-0.31168', '-0.038396', '-0.13885', '0.13174', '0.48543',
        '0.27012', '0.023631', '0.039914', '-0.0085227', '-0.32833',
        '-0.34518', '-0.039726', '-0.19771', '0.23373', '-0.041411',
        '-0.036932', '-0.073936', '-0.47071', '0.30993', '0.32195',
        '-0.074116', '0.17782', '-0.33701', '0.12694', '0.07969',
        '-0.078333', '-0.32187', '-0.21694', '-0.32914', '0.38804',
        '-0.47941', '0.52507', '0.13306', '-0.086922', '-0.1153',
        '0.64853', '-0.28492', '0.55657', '0.037141', '-0.24895',
        '0.26131', '-0.094953', '0.061678', '0.19854', '0.22078',
        '0.27887', '-0.65879', '-0.03955', '0.62236', '-0.22495',
        '-0.056741', '0.017541', '0.072202', '0.017473'

In [98]:
lyrics_df['embedding'] = lyrics_df['lyrics'][:30].apply(lambda x : merge_embedding_pos(x))

/bin/bash: -c: line 0: unexpected EOF while looking for matching `''
/bin/bash: -c: line 0: unexpected EOF while looking for matching `''
/bin/bash: -c: line 0: unexpected EOF while looking for matching `''
/bin/bash: -c: line 0: unexpected EOF while looking for matching `''
/bin/bash: -c: line 0: unexpected EOF while looking for matching `''
/bin/bash: -c: line 0: unexpected EOF while looking for matching `''
/bin/bash: -c: line 0: unexpected EOF while looking for matching `''
/bin/bash: -c: line 0: unexpected EOF while looking for matching `''
/bin/bash: -c: line 0: unexpected EOF while looking for matching `''
/bin/bash: -c: line 0: unexpected EOF while looking for matching `''
/bin/bash: -c: line 0: unexpected EOF while looking for matching `''
/bin/bash: -c: line 0: unexpected EOF while looking for matching `''
/bin/bash: -c: line 0: unexpected EOF while looking for matching `''
/bin/bash: -c: line 0: unexpected EOF while looking for matching `''
/bin/bash: -c: line 0: unexpected 

Can we remove SYM for symbol, PUNCT for punctuation, and/or merge some categories ?

In [107]:
lyrics_df.head(10)

,artist,genre,lyrics,title,embedding
0,beyonce-knowles,pop,Oh baby how you doing\nYou know I'm gonna cut ...,ego-remix,"[[Oh, -0.26756, 0.16066, -0.026644, -0.061647,..."
1,beyonce-knowles,pop,playin' everything so easy\nit's like you seem...,then-tell-me,"[[playin', 0.046408, -0.22275, 0.27501, 0.3307..."
2,beyonce-knowles,pop,If you search\nFor tenderness\nIt isn't hard t...,honesty,"[[If, -0.077863, 0.26915, 0.24783, 0.078624, 0..."
3,beyonce-knowles,pop,Oh oh oh I oh oh oh I\n\nIf I wrote a book abo...,you-are-my-rock,"[[Oh, -0.26756, 0.16066, -0.026644, -0.061647,..."
4,beyonce-knowles,pop,Party the people the people the party it's pop...,black-culture,"[[Party, -0.031328, -0.60994, -0.27762, -0.313..."
5,beyonce-knowles,pop,I heard\nChurch bells ringing\nI heard\nA choi...,all-i-could-do-was-cry,"[[I, -0.028172, 0.063204, 0.13383, 0.013919, -..."
6,beyonce-knowles,pop,This is just another day that I would spend\nW...,once-in-a-lifetime,"[[This, 0.026437, 0.014923, 0.17864, -0.10028,..."
7,beyonce-knowles,pop,Waiting waiting waiting waiting\nWaiting waiti...,waiting,"[[Waiting, -0.36918, 0.38065, 0.68427, -0.1569..."
8,beyonce-knowles,pop,\nI read all of the magazines\nwhile waiting a...,slow-love,"[[I, -0.028172, 0.063204, 0.13383, 0.013919, -..."
9,beyonce-knowles,pop,Nnnow honey\nYou better sit down and look arou...,why-don-t-you-love-me,"[[Nnnow, 0.014805, -0.076101, -0.017499, -0.13..."


## 6. Create column with Entity Recognition

In [122]:
nlp = spacy.load('en_core_web_sm')

Entity types : <br>
PERSON	People, including fictional.<br>
NORP	Nationalities or religious or political groups.<br>
FAC	Buildings, airports, highways, bridges, etc.<br>
ORG	Companies, agencies, institutions, etc.<br>
GPE	Countries, cities, states.<br>
LOC	Non-GPE locations, mountain ranges, bodies of water.<br>
PRODUCT	Objects, vehicles, foods, etc. (Not services.)<br>
EVENT	Named hurricanes, battles, wars, sports events, etc.<br>
WORK_OF_ART	Titles of books, songs, etc.<br>
LAW	Named documents made into laws.<br>
LANGUAGE	Any named language.<br>
DATE	Absolute or relative dates or periods.<br>
TIME	Times smaller than a day.<br>
PERCENT	Percentage, including "%".<br>
MONEY	Monetary values, including unit.<br>
QUANTITY	Measurements, as of weight or distance.<br>
ORDINAL	"first", "second", etc.<br>
CARDINAL	Numerals that do not fall under another type.<br>

In [136]:
doc = nlp(lyrics_df.loc[24]['lyrics'].replace('\n', '').lower())
for ent in doc.ents:
    print(ent.text, ent.label_)

the first day DATE
first ORDINAL
the first day DATE
the first day DATE
the first day the first day DATE
the first day DATE
the first day DATE
the first day DATE


In [154]:
def find_sub_list(sl,l):
    sll=len(sl)
    for ind in (i for i,e in enumerate(l) if e==sl[0]):
        if l[ind:ind+sll]==sl:
            return ind,ind+sll-1

In [195]:
def get_entity_for_lyrics(l):
    l = l.replace('\n', ' ').lower()
    doc = nlp(l)
    l_list = l.split(' ')
    curr_i = 0
    result = np.empty(shape=len(l_list), dtype=object)
    for ent in doc.ents:
        if ent.text.replace(' ', '') != '':
            ent_list = ent.text.split(' ')
            try:
                s, e = find_sub_list(ent_list, l_list[curr_i:])
                s += curr_i
                e += curr_i
                for i in range(s, e+1):
                    result[i] = ent.label_
                curr_i = e
            except : 
                print(ent_list)
                print(l_list[curr_i:])
    return result

In [159]:
ent_to_idx = {'PERSON' : 0,
'NORP': 1,
'FAC' : 2,
'ORG' : 3,
'GPE' : 4,
'LOC' : 5,
'PRODUCT' : 6,
'EVENT' : 7,
'WORK_OF_ART' : 8,
'LAW': 9,
'LANGUAGE' : 10,
'DATE' : 11,
'TIME' : 12,
'PERCENT' : 13,
'MONEY' : 14,
'QUANTITY' : 15,
'ORDINAL' : 16,
'CARDINAL': 17}

In [169]:
def add_ent_to_embedding(emb, l):
    # Compute the entities for the lyrics
    ents = get_entity_for_lyrics(l)
    # Go over the embedding of the words
    for i, w in enumerate(emb):
        # Create empty binary vector
        bin_vector = np.zeros(18)
        # If the entity is None, means we don't know --> Keep all 0s
        if ents[i] != None :
            # Put 1 where the entity is when we know it : current entity is ents[i], pass to index with ent_to_idx
            bin_vector[ent_to_idx[ents[i]]] = 1
        # Add the binary vector at the end of embedding and return
        emb[i] = np.append(w, bin_vector)
    return emb

In [171]:
add_ent_to_embedding(lyrics_df.loc[0]['embedding'], lyrics_df.loc[0]['lyrics'])

[array(['Oh', '-0.26756', '0.16066', '-0.026644', '-0.061647', '0.12856',
        '-0.16667', '-0.31698', '0.21519', '0.087186', '0.70611',
        '0.097478', '-0.017475', '0.13562', '-0.34733', '0.12901',
        '0.060213', '0.0038623', '-0.003505', '-0.309', '-0.1214',
        '-0.31168', '-0.038396', '-0.13885', '0.13174', '0.48543',
        '0.27012', '0.023631', '0.039914', '-0.0085227', '-0.32833',
        '-0.34518', '-0.039726', '-0.19771', '0.23373', '-0.041411',
        '-0.036932', '-0.073936', '-0.47071', '0.30993', '0.32195',
        '-0.074116', '0.17782', '-0.33701', '0.12694', '0.07969',
        '-0.078333', '-0.32187', '-0.21694', '-0.32914', '0.38804',
        '-0.47941', '0.52507', '0.13306', '-0.086922', '-0.1153',
        '0.64853', '-0.28492', '0.55657', '0.037141', '-0.24895',
        '0.26131', '-0.094953', '0.061678', '0.19854', '0.22078',
        '0.27887', '-0.65879', '-0.03955', '0.62236', '-0.22495',
        '-0.056741', '0.017541', '0.072202', '0.017473'

In [196]:
for i, r in lyrics_df[:30].iterrows() :
    r['embedding'] = add_ent_to_embedding(r['embedding'], r['lyrics'])

['night']
['one', 'more', 'dance', 'then', "i'm", 'ready', 'to', 'go', 'well', 'the', "dj's", 'playing', 'all', 'the', 'same', 'songs', 'and', 'the', "night's", 'about', 'to', 'end', 'can', 'we', 'meet', 'in', 'the', 'parking', 'lot', 'find', 'a', 'quiet', 'place', 'were', 'we', 'can', 'talk', 'to', 'find', 'out', 'more', 'about', 'each', 'other', 'baby', 'can', 'we', 'repeat', 'prechorus', 'and', 'chorus', 'bridge', 'you', 'know', "i'm", 'feelin', 'you', 'tonight', 'so', "let's", 'find', 'a', 'certain', 'spot', 'to', 'go', 'where', 'we', 'can', 'get', 'to', 'know', 'each', 'other', 'better', "i'll", 'go', 'and', 'tell', 'my', 'girls', 'you', 'go', 'and', 'tell', 'your', 'boys', 'before', 'we', "leavin'", 'tell', 'me', 'whats', "it'", "gon'", 'be', '', 'chorus', 'until', 'fade']


Wikipedia scheme : <br>
PER	Named person or family.<br>
LOC	Name of politically or geographically defined location (cities, provinces, countries, international regions, bodies of water, mountains).<br>
ORG	Named corporate, governmental, or other organizational entity.<br>
MISC	Miscellaneous entities, e.g. events, nationalities, products or works of art.<br>

In [111]:
#with wikipedia scheme
nlp_wiki = spacy.load('xx_ent_wiki_sm')

In [120]:
doc = nlp_wiki(lyrics_df.loc[24]['lyrics'].replace('\n', '').lower())
for ent in doc.ents:
    print(ent, ent.label_)